# **Custumer Segmentation And AI Recomendation**

In [ ]:
#Name : Deore Siddhi Arun
#Internship : AI Intern

In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn plotly --quiet



In [ ]:
 import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn s
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
import zipfile, os


In [ ]:
# In Colab, run this and upload:
# - events.csv.zip
# - item_properties_part1.zip
# - item_properties_part2.zip
# - category_tree.csv

from google.colab import files
uploaded = files.upload()


Saving category_tree.csv to category_tree (2).csv
Saving events.csv.zip to events.csv.zip
Saving item_properties_part1.csv.zip to item_properties_part1.csv.zip
Saving item_properties_part2.csv.zip to item_properties_part2.csv.zip


In [ ]:
from google.colab import files

# This will prompt you to upload both zip files
uploaded = files.upload()


In [ ]:
import zipfile
import os

# Optional: check files in current directory
print("Uploaded files:", os.listdir())

# Unzip events
with zipfile.ZipFile("events.csv.zip", "r") as z:
    z.extractall("events")  # Extract into 'events' folder

# Unzip item_properties parts using exact names
with zipfile.ZipFile("item_properties_part1.csv (1).zip", "r") as z:
    z.extractall("item_properties_part1")  # Extract into folder

with zipfile.ZipFile("item_properties_part2.csv (1).zip", "r") as z:
    z.extractall("item_properties_part2")  # Extract into folder

# Check extracted folders
print("Events files:", os.listdir("events"))
print("Item Properties Part 1 files:", os.listdir("item_properties_part1"))
print("Item Properties Part 2 files:", os.listdir("item_properties_part2"))


Uploaded files: ['.config', 'item_properties_part2.csv.zip', 'events.csv', 'category_tree (1).csv', 'item_properties_part1.csv.zip', 'category_tree.csv', 'events.csv.zip', 'category_tree (2).csv', 'item_properties_part2.csv (1).zip', 'item_properties_part1.csv (1).zip', 'sample_data']
Events files: ['events.csv']
Item Properties Part 1 files: ['item_properties_part1.csv']
Item Properties Part 2 files: ['item_properties_part2.csv']


In [ ]:


# Load item_properties CSVs from their folders
item1 = pd.read_csv("item_properties_part1/item_properties_part1.csv")
item2 = pd.read_csv("item_properties_part2/item_properties_part2.csv")

# Combine both parts
item_properties = pd.concat([item1, item2], ignore_index=True)

# Save the merged CSV
item_properties.to_csv("item_properties.csv", index=False)

# Optional: preview first few rows
print(item_properties.head())


       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566
3  1431226800000   59481         790                       n15360.000
4  1431831600000  156781         917                           828513


In [ ]:
events = pd.read_csv("events.csv")
item_properties = pd.read_csv("item_properties.csv")
category_tree = pd.read_csv("category_tree.csv")

print("Events shape:", events.shape)
print("Item properties shape:", item_properties.shape)
print("Category tree shape:", category_tree.shape)


Events shape: (2756101, 5)
Item properties shape: (20275902, 4)
Category tree shape: (1669, 2)


In [ ]:
# Filter only purchase (transaction) events
transactions = events[events['event'] == 'transaction']

# Merge with category info
item_props_filtered = item_properties[item_properties['property'] == 'categoryid']
transactions = transactions.merge(item_props_filtered, on="itemid", how="left")

# Clean missing values
transactions = transactions.dropna()
transactions.head()


,timestamp_x,visitorid,event,itemid,transactionid,timestamp_y,property,value
0,1433222276276,599528,transaction,356475,4000.0,1.433041e+12,categoryid,386
1,1433193500981,121688,transaction,15335,11117.0,1.431227e+12,categoryid,1098
2,1433193915008,552148,transaction,81345,5444.0,1.431832e+12,categoryid,1393
3,1433176736375,102019,transaction,150318,13556.0,1.433041e+12,categoryid,1236
4,1433174518180,189384,transaction,310791,7244.0,1.431832e+12,categoryid,437


In [ ]:
customer_features = transactions.groupby("visitorid").agg({
    "itemid": "count",
    "transactionid": "nunique",
}).reset_index()

customer_features.rename(
    columns={"itemid": "total_items", "transactionid": "unique_transactions"},
    inplace=True
)

customer_features.head()



,visitorid,total_items,unique_transactions
0,172,2,1
1,186,6,1
2,264,2,1
3,419,1,1
4,539,1,1


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(customer_features[["total_items", "unique_transactions"]])

kmeans = KMeans(n_clusters=3, random_state=42)
customer_features['cluster'] = kmeans.fit_predict(X_scaled)

print("Silhouette Score (KMeans):", silhouette_score(X_scaled, customer_features['cluster']))


Silhouette Score (KMeans): 0.9790726092677798


In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
customer_features['dbscan_cluster'] = dbscan.fit_predict(X_scaled)


In [ ]:
fig = px.scatter(
    customer_features,
    x="total_items",
    y="unique_transactions",
    color="cluster",
    title="Customer Segments (KMeans)",
    template="plotly_dark"
)
fig.show()


In [ ]:
fig = px.scatter(
    customer_features,
    x="total_items",
    y="unique_transactions",
    color="cluster",
    title="Customer Segments (KMeans)",
    template="plotly_dark"
)
fig.show()


In [ ]:
basket = transactions.groupby(['visitorid', 'itemid']).size().unstack(fill_value=0)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(basket.values)

visitor_id = customer_features['visitorid'].iloc[0]
visitor_vector = basket.loc[visitor_id].values.reshape(1, -1)

distances, indices = model_knn.kneighbors(visitor_vector, n_neighbors=5)
print(f"Recommendations for visitor {visitor_id}:")
print(basket.index[indices.flatten()].tolist())


Recommendations for visitor 172:
[172, 1041255, 883121, 496310, 6855]


In [ ]:
print("\nCluster Insights:")
for c in customer_features['cluster'].unique():
    group = customer_features[customer_features['cluster'] == c]
    print(f"Cluster {c}: Avg Items = {group['total_items'].mean():.2f}, Avg Transactions = {group['unique_transactions'].mean():.2f}")



Cluster Insights:
Cluster 0: Avg Items = 3.12, Avg Transactions = 1.20
Cluster 1: Avg Items = 159.82, Avg Transactions = 73.35
Cluster 2: Avg Items = 745.00, Avg Transactions = 324.33
